<a href="https://colab.research.google.com/github/Rajeeb321123/Task_for_llm/blob/main/idefics_2_FT_multimodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install --upgrade pip -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.8 MB/s eta 0:00:00


In [ ]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q  bitsandbytes peft hf_transfer flash_attn
!pip install -q -U git+https://github.com/huggingface/peft.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 42.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 

In [ ]:
!pip freeze > requirement.txt

In [ ]:
# Push the model later on
from huggingface_hub import notebook_login

notebook_login()

----------------------------------

## Loading the model and the dataset

- model = `idefics2-8b`
- finetuning dataset = samll chess dataset

In [ ]:
# Enable falst weights download and upload using rust library
%env HF_ENABLE_HF_TRANSFER =1

env: HF_ENABLE_HF_TRANSFER=1


In [ ]:
import torch
from peft import LoraConfig, get_peft_model
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics2ForConditionalGeneration

Device = "auto" # loading to one GPU
USE_LORA = False # for higher Gpu
USE_QLORA = True # for lower Gpu
dtype = torch.float16 # torch.bfloat16 for Higher gpu like A100, bfloat16 retain no of weights in float32 but float16 is unstable.


In [ ]:
# We don't need tokenizer processor handle both image and text. similar to AutoTokenizer
processor = AutoProcessor.from_pretrained(
    'HuggingFaceM4/idefics2-8b',
    # image splitting inputs single image along with its splited small subset of image
    # if false only single image
    do_image_slitting = False #set true for higher quality. Need 55+ GPU
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bits = True,
    bnb_4bit_quant_type ="nf4",
    bnb_4bit_compute_dtype = dtype
)

# Note that weights are in 32 bits, so quantization is needed for lower gpu like me
model = Idefics2ForConditionalGeneration.from_pretrained(
    "HuggingFaceM4/idefics2-8b",
    torch_dtype = dtype,
    device_map = Device,
    # _attn_implmentation = "flash_attention_2" # only not using if USE_QLORA  else put here None, It require A100 or h100
    quantization_config = bnb_config, # if Use_QLORA else None
    cache_dir = '' # to save weights in current working dir
)

Unused kwargs: ['load_in_4bits']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/74.4k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.64G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/4.25G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
# check there are no parameters overflowing onto cpu (meta).
for n, p in model.named_parameters():
  if p.device.type == 'meta':
    print(f"{n} is on meta!")
  elif p.device.type == "cpu":
    print(f"{n} is on cpu")

In [ ]:
print(model) # very imp for finding target modules to choose for lora config below

Idefics2ForConditionalGeneration(
  (model): Idefics2Model(
    (vision_model): Idefics2VisionTransformer(
      (embeddings): Idefics2VisionEmbeddings(
        (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
        (position_embedding): Embedding(4900, 1152)
      )
      (encoder): Idefics2Encoder(
        (layers): ModuleList(
          (0-26): 27 x Idefics2EncoderLayer(
            (self_attn): Idefics2VisionAttention(
              (k_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (v_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (q_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
              (out_proj): Linear4bit(in_features=1152, out_features=1152, bias=True)
            )
            (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
            (mlp): Idefics2VisionMLP(
              (activation_fn): PytorchGELUTanh()
              

In [ ]:
# # try out left padding can (may) lead for beter reponse
# processor.tokenizer.padding_side = 'left'

-----------------------------------------------------

## Lora Config for Training later on

* 3 options for training:
  - QloRA
  - Standard LoRA
  - full fine-tuning

In [ ]:
# if USE_QLORA or USE_LORA:
#   lora_config = LoraConfig(
#       #rank:r: les means less paramters training
#       r = 8,  # for more granularity use 32
#       lora_alpha = 32, # for near 7b parameter model
#       lora_dropout = 0.1,

#       # here we are not fine tunning vision model. cost: VRAM
#       # if low GPU just preciever_resampler and its layers but we did text_model and modality_projection as well
#       #down up k q.. means mlp layers and attention layer with in module of text_model|modality_projection|preceiver_resampler
#       target_modules = '.*(text_model|modality_projection|perceiver_resampler).*(down_proj|gate_proj|up_proj|k_proj|q_proj|v_proj|o_proj).*$',
#       use_dora = False if USE_QLORA else True,
#       init_lora_weights = "gaussian",
#       use_rslora = True # rank stablized lora: training speed based on rank size. we can larger steps on lower dim space (small paramters no) but high dim space: more intrigate steps.
#   )
#   model.add_adapter(lora_config)
#   model.enable_adapters() #By using adapters, you can fine-tune only the relevant parts of the model, reducing the risk of overfitting and saving memory.
# else:
#   print("No adapters being added for full fine-tunning")

if USE_QLORA or USE_LORA:
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules='.*(text_model|modality_projection|perceiver_resampler).*(down_proj|gate_proj|up_proj|k_proj|q_proj|v_proj|o_proj).*$',
        use_dora=False if USE_QLORA else True,
        init_lora_weights="gaussian"
    )
    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
    model.add_adapter(lora_config)
    model.enable_adapters()
else:
    model = Idefics2ForConditionalGeneration.from_pretrained(
        "HuggingFaceM4/idefics2-8b",
        torch_dtype=torch.float16,
        _attn_implementation="flash_attention_2", # This works for A100 or H100
    ).to(Device)




## Load Dataset

In [ ]:
from datasets import load_dataset

# load and prepare dataset
ds = load_dataset("Trelis/chess_pieces")

# train and eval split
train_dataset = ds["train"]
eval_dataset = ds["test"]

Generating train split:   0%|          | 0/48 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3 [00:00<?, ? examples/s]

In [ ]:
train_dataset[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3024x4032>,
 'caption': 'A single white rook.'}

In [ ]:
# train_dataset[10]["image"]

## Evaluation before training

How will model perform without training on dataset

In [ ]:
import torch
from PIL import Image
import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
from torchvision.transforms.functional import to_pil_image, resize

def run_model_evaluation(dataset, num_samples = None, device ='cuda', constant_query = None):
  model.eval()
  result = []

  # limit the dataset if a specific numbers of samples is provided
  if num_samples is not None:
    dataset = torch.utils.data.Subset(dataset, range(num_samples))

  plt.figure()
  i = 0
  for example in dataset:
    image = example["image"]
    if constant_query is None:
      query = example["query"]["en"]
    else:
      query = constant_query # Use the constant query if provided

    # Display a reduced size version of the image
    pil_image = image
    aspect_ratio = pil_image.width / pil_image.height
    new_width = 300
    new_height = int(new_width/aspect_ratio)
    display_image = resize(pil_image, (new_height, new_width))
    plt.subplot(1, 2, i)
    i = i+1
    plt.imshow(display_image)
    plt.axis("off")
    # Construct the message template
    messages = [
        {
            "role": "user",
            "content": [
                    {"type": "image"},
                    {"type": "text", "text":query}
            ]
            }
    ]

    # Apply the chat template to preprocess input
    text = processor.apply_chat_template(messages, add_generation_prompt = True)
    inputs = processor(text=[text.strip()], images=[image], return_tensors="pt", add_special_tokens=False)

    # Generate output from the model
    generated_ids = model.generate(**inputs, max_new_tokens=64)
    generated_texts = processor.batch_decode(generated_ids[:, inputs["input_ids"].size(1):], skip_special_tokens=True)

    print(f"prediction: {generated_texts[0]} \n")

    result.append(generated_texts[0]) # store the results

  return result
# To do for multiple images

In [ ]:
# Usage
eval_results = run_model_evaluation(eval_dataset, num_samples=3, device='cuda', constant_query = "What kind of chess piece do you see")

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

4149

## Training Loop

- define data collator: take list of samples and return input tensor fed to the model.
- 4 tensors types we are interested in:
  - `input_ids`: input indexes fed to the language model.
  - `attention_mask`: the attention mask for the input_ids in llm
  - `pixel_values`: the pre-processed pixel values that encode the image(s). Idefics2 treat images in their native resolution (up to 980) and native aspect ratio
  - `pixel_attention_mask`: when multiple images are packed into same sample (or in batch), attention masks for images are necessary because of these images can have different sizes and aspect ratio. This masking ensures that vision encoder properly forwards the images.

In [ ]:
a = [1,2,3,1,4,5,1]
a [ a == 1] = 0
print(a)

[0, 2, 3, 1, 4, 5, 1]


In [ ]:
# import random
# # https://huggingface.co/docs/transformers/en/main_classes/data_collator

# class MyDataCollator: # collator means collect + combine
#   def __init__(self, processor):
#     self.processor = processor
#     self.image_token_id = processor.tokenizer.additional_special_tokens_ids[
#         processor.tokenizer.additional_special_tokens.index("<image>")
#     ]
#   def __call__(self, examples):
#     texts = []
#     images = []
#     for example in examples:
#       image = example["image"]
#       question = 'What do you see here' # for chess dataset
#       answer = example["caption"] # for chess dataset
#       messages = [
#           {
#               "role": "user",
#               "content": [
#                   # {"type": "text", "text": "Answer briefly."},
#                   {"type": "image"},
#                   {"type": "text", "text": question}
#               ]
#           },
#           # we also provide assistant output for fine tunning
#           {
#               "role": "assistant",
#               "content": [
#                   {"type": "text", "text":answer}
#               ]
#           }
#       ]
#       text = processor.apply_chat_template(messages, add_generation_prompt=False)
#       texts.append(text.strip())
#       images.append([image])

#     batch = processor(text=texts, images=images, return_tensors="pt", padding=True) # pade them all to same length
#     print("batch:", batch)

#     # create labels
#     # way to specify wher will be image , model will get use to where will be the images
#     # there will be shifting happen internally
#     labels = batch["input_ids"].clone()
#     print("labels:",labels)
#     labels[labels == processor.tokenizer.pad_token_id] = self.image_token_id
#     print("labels:",labels)
#     batch["labels"] = labels
#     print("batch:",batch)

#     return batch

# data_collator = MyDataCollator(processor)
# # TO do for multiple images at once

import random

# DataCollator
class MyDataCollator:
    def __init__(self, processor):
        self.processor = processor
        self.image_token_id = processor.tokenizer.additional_special_tokens_ids[
            processor.tokenizer.additional_special_tokens.index("<image>")
        ]

    def __call__(self, examples):
        texts = []
        images = []
        for example in examples:
            texts = []
            images = []
            for example in examples:
              image = example["image"]
              question = 'What do you see here' # for chess dataset
              answer = example["caption"] # for chess dataset
              messages = [
                  {
                      "role": "user",
                      "content": [
                          # {"type": "text", "text": "Answer briefly."},
                          {"type": "image"},
                          {"type": "text", "text": question}
                      ]
                  },
                  # we also provide assistant output for fine tunning
                  {
                      "role": "assistant",
                      "content": [
                          {"type": "text", "text":answer}
                      ]
                  }
              ]
            text = processor.apply_chat_template(messages, add_generation_prompt=False)
            texts.append(text.strip())
            images.append([image])

        batch = processor(text=texts, images=images, return_tensors="pt", padding=True)

        labels = batch["input_ids"].clone()
        labels[labels == processor.tokenizer.pad_token_id] = self.image_token_id
        batch["labels"] = labels

        return batch

# DataCollator
data_collator = MyDataCollator(processor)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    # # max_steps = 1, # how max_steps affect: https://discuss.huggingface.co/t/how-does-max-steps-affect-the-number-of-samples-the-model-sees/69681
    # num_train_epochs = 2,
    # per_device_train_batch_size = 1,
    # per_device_eval_batch_size = 1,
    # gradient_accumulation_steps =1, # if batch size was 4, gradient accumulation should be 8. 4X8 =32 (virtual size = 32) which give nice averaging accross each batch: smooth training
    # # warmup_steps = 50, #comment in only if more than 50 samples
    # learning_rate = 1e-5, # setting a bit lower as rs-lora is already used which accelerate the learning
    # weight_decay = 0.01,
    # logging_steps = 5,
    # output_dir = "fine-tuned-model",
    # # eval_strategy = "steps",
    # # eval_steps = 0.25, # evaluate every quarter of no of steps
    # #save_stragey = "steps",
    # #save_steps = 250,
    # #evalution_strategy="epoch",
    # fp16 = True,
    # # bf16 = True,
    # # hub_model_id = "Rajeeb321/idefics2-8b-image-test," # hub i want to push to after training
    # # report_to = "wandb" for mlops need wandb account
    # gradient_checkpointing = True, # reduce VRam requirement
    # gradient_checkpointing_kwargs = {'use_reentrant': True}
    num_train_epochs=2,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=8,
    warmup_steps=50,
    learning_rate = 1e-4,
    weight_decay=0.01,
    logging_steps=25,
    output_dir = "/content",
    save_strategy = "steps",
    save_steps = 25,
    save_total_limit = 1,
    fp16 = True,
    remove_unused_columns=False,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args = training_args,
    data_collator = data_collator,
    train_dataset = train_dataset,
    # eval_dataset= eval_dataset, # You can also evaluate (loss) to eval set,,
)

In [ ]:
len(train_dataset)

48

In [ ]:
eval_dataset[1]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3024x4032>,
 'caption': 'A black rook, a black bishop and a white bishop all standing together.'}

## Training and pushing to Hub

In [ ]:
trainer.train() # idealy larger batch size , larger training steps per epoch, lower lr, but VRAM needed

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.95 GiB. GPU 

## OCR evauation

- use anls: